In [ ]:
# === Imports ===
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# === LEITURA DO CSV E AJUSTES ===
df = pd.read_csv("dados_rios_ficticios.csv", encoding="utf-8", sep=";")
df.columns = [col.encode('utf-8').decode('utf-8-sig').strip().lower() for col in df.columns]

for col in df.columns:
    if "data" in col:
        df.rename(columns={col: "data"}, inplace=True)
        break

df["data"] = pd.to_datetime(df["data"], format="%d/%m/%Y", errors="coerce")

# === CATEGORIZAÇÃO PARA VISUALIZAÇÃO ===
def classificar_nivel(nivel):
    if nivel < 5.5:
        return "Normal"
    elif 5.5 <= nivel < 7.5:
        return "Atenção"
    else:
        return "Risco"

df["nivel_categoria"] = df["nivel_metro"].apply(classificar_nivel)

# === ANÁLISE INICIAL ===
print("Dados ausentes por coluna:")
print(df.isnull().sum())

print("\nEstatísticas descritivas:")
print(df.describe())

# === GERAÇÃO DE GRÁFICOS ===

# 1. Distribuição de categorias de risco
plt.figure(figsize=(6, 4))
df["nivel_categoria"].value_counts().plot(kind="bar")
plt.title("Distribuição de Risco do Nível do Rio")
plt.xlabel("Categoria")
plt.ylabel("Número de Ocorrências")
plt.tight_layout()
plt.savefig("distribuicao_risco.png")
plt.show()

# 2. Chuva vs Nível do Rio
plt.figure(figsize=(6, 4))
sns.scatterplot(data=df, x="chuva_mm", y="nivel_metro", hue="nivel_categoria")
plt.title("Chuva vs Nível do Rio")
plt.xlabel("Chuva (mm)")
plt.ylabel("Nível do Rio (m)")
plt.tight_layout()
plt.savefig("chuva_vs_nivel_categoria.png")
plt.show()

# 3. Média de Vazão por Categoria
plt.figure(figsize=(6, 4))
df.groupby("nivel_categoria")["vazao_m3s"].mean().plot(kind="bar")
plt.title("Média de Vazão por Categoria de Risco")
plt.ylabel("Vazão Média (m³/s)")
plt.tight_layout()
plt.savefig("media_vazao_por_categoria.png")
plt.show()

# 4. Mapa de correlação entre variáveis numéricas
plt.figure(figsize=(6, 4))
sns.heatmap(df[["nivel_metro", "chuva_mm", "vazao_m3s"]].corr(), annot=True, cmap="coolwarm")
plt.title("Correlação entre Variáveis")
plt.tight_layout()
plt.show()
